<a href="https://colab.research.google.com/github/DeboraOliver/MachineLearning_lessons/blob/main/Redes_Neurais_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.youtube.com/watch?v=QIUxPv5PJOY
# Let's use an artificial recurrent neural network called Long Short Term Memory (LSTM) to predict the closing stock price of a corporation using the past 60 day stock price

In [ ]:
!pip install -q yfinance

In [ ]:
import math
import pandas as pd
import pandas_datareader as web
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
plt.style.use('fivethirtyeight')


In [ ]:
#Get the stock quote

start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2022, 5, 10)
df = yf.download('TOTS3.SA', start = start, end=end)
df

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
# plt.xlable("Date",fontsize=18)
plt.ylable('Close Price USD', fontsize=18)
plt.show()

In [ ]:
#creating a new df with only the "Close' column

data = df.filter(['Close'])
dataset = data.values #convert to numpy array

#get the number of rows to train the model
training_data_len = math.ceil(len(dataset) * .8)
training_data_len

In [ ]:
#Scaled the data: normalization is a good price that helps the model
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
#create the traininf data set
#create the scaled training data ser
train_data = scaled_data[0:training_data_len , :]
#split into x_train and y tain
x_train =[]
y_train = []
for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])
  if i<= 60:
    print(x_train,y_train)


In [ ]:
# convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

#Reshape the data because LSTM need a 3 dimentional and for now we only have a 2 dimentional object
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
from pickle import FALSE
#Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Compile the model
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
#train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
#create the testing data
#create a new array containing scaled values from index 1542 to 2003
test_data = scaled_data[training_data_len - 60: , :]
#Create the data sets x_test and y test
x_test = []
y_test = dataset[training_data_len:, :]

for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

In [ ]:
#convert the data to a numpy array
x_test = np.array(x_test)

#Reshape data to a 3 dimention shape
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

#get the models predict price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions) #unscaling

In [ ]:
#get the root men square error (RMSE (lower values is good))
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
from matplotlib import figure
#plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('Model')
plt.ylabel('Close Price in USD', fontsize = 18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train','Val','Predictions'], loc = 'lower right')
plt.show()

In [ ]:
#show the valid and predicted prices
valid

In [ ]:
#get the quote
apple_quote = yf.download('TOTS3.SA', start = start, end='2022-05-12')
#create dataframe
new_df = apple_quote.filter(['Close'])
#Get the 60 day closing price values and convert the dataframe to an array
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
# #append the past 60 days
X_test.append(last_60_days_scaled)
# #convert the x_test data set to a numpy array
X_test = np.array(X_test)
# #Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
#Get the predict scaled price
pred_price = model.predict(X_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
#get the quote
apple_quote2 = yf.download('TOTS3.SA', start = '2022-05-13', end='2022-05-14')
print(apple_quote2['Close'])

In [ ]:
X_test